## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
file_name= os.path.join('..','Weather_Database','WeatherPy_database.csv')
city_data_df = pd.read_csv(file_name)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Khatanga,RU,71.9667,102.5000,37.24,91,100,10.80,overcast clouds
1,Dingle,PH,10.9995,122.6711,78.78,87,87,2.17,light rain
2,Mataura,NZ,-46.1927,168.8643,49.48,99,100,0.83,light rain
3,Lebu,CL,-37.6167,-73.6500,55.31,82,100,8.34,overcast clouds
4,Boguchany,RU,58.3814,97.4531,37.51,97,100,10.94,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,Dingle,PH,10.9995,122.6711,78.78,87,87,2.17,light rain
9,Georgetown,MY,5.4112,100.3354,82.35,88,20,1.99,few clouds
10,Manggar,ID,-2.8833,108.2667,81.95,77,68,4.12,broken clouds
11,Euclides+Da+Cunha,BR,-10.5075,-39.0158,85.96,28,8,8.34,clear sky
12,Cap+Malheureux,MU,-19.9842,57.6142,73.89,78,19,3.00,few clouds
14,Salinopolis,BR,-0.6136,-47.3561,84.94,64,17,8.63,few clouds
17,Rikitea,PF,-23.1203,-134.9692,73.96,71,69,17.11,broken clouds
18,Santa+Fe,AR,-31.6333,-60.7000,74.75,52,55,5.01,broken clouds
20,Kapaa,US,22.0752,-159.3190,75.52,83,40,9.33,light rain
24,Shizuishan,CN,39.2333,106.7694,73.36,14,63,10.67,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   293
Country                293
Lat                    293
Lng                    293
Max Temp               293
Humidity               293
Cloudiness             293
Wind Speed             293
Current Description    293
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# NO EMPTY ROWS

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Dingle,PH,78.78,light rain,10.9995,122.6711,
9,Georgetown,MY,82.35,few clouds,5.4112,100.3354,
10,Manggar,ID,81.95,broken clouds,-2.8833,108.2667,
11,Euclides+Da+Cunha,BR,85.96,clear sky,-10.5075,-39.0158,
12,Cap+Malheureux,MU,73.89,few clouds,-19.9842,57.6142,
14,Salinopolis,BR,84.94,few clouds,-0.6136,-47.3561,
17,Rikitea,PF,73.96,broken clouds,-23.1203,-134.9692,
18,Santa+Fe,AR,74.75,broken clouds,-31.6333,-60.7000,
20,Kapaa,US,75.52,light rain,22.0752,-159.3190,
24,Shizuishan,CN,73.36,broken clouds,39.2333,106.7694,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# 6d. Set up the base URL for the Google Directions API to get JSON data.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    try:
        # 6c. Get latitude and longitude from DataFrame
        lat = row["Lat"]
        lng = row["Lng"]
        params["location"] = f"{lat},{lng}"
        #new_url = base_url+"location="+Lat+","+Lng+"&radius=5000&types=lodging&key="+g_key
    # 6e. Make request and retrieve the JSON data from the search. 
        hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.") 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
#hotel_df.dropna(subset=["Hotel Name"])
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!="")]

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

In [12]:
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))